In [1]:
sc

In [2]:
import csv

nypd_collisions = sc.textFile('nypd_carl2_part0.csv')
weather_data = sc.textFile('processed_2011-2017_final.csv')

In [3]:
list(enumerate(nypd_collisions.first().split(',')))

[(0, u'BOROUGH'),
 (1, u'CONTRIBUTING_FACTOR_VEHICLE_1'),
 (2, u'CONTRIBUTING_FACTOR_VEHICLE_2'),
 (3, u'CONTRIBUTING_FACTOR_VEHICLE_3'),
 (4, u'CONTRIBUTING_FACTOR_VEHICLE_4'),
 (5, u'CONTRIBUTING_FACTOR_VEHICLE_5'),
 (6, u'CROSS_STREET_NAME'),
 (7, u'DATETIME'),
 (8, u'LATITUDE'),
 (9, u'LOCATION'),
 (10, u'LONGITUDE'),
 (11, u'NUMBER_OF_CYCLIST_INJURED'),
 (12, u'NUMBER_OF_CYCLIST_KILLED'),
 (13, u'NUMBER_OF_MOTORIST_INJURED'),
 (14, u'NUMBER_OF_MOTORIST_KILLED'),
 (15, u'NUMBER_OF_PEDESTRIANS_INJURED'),
 (16, u'NUMBER_OF_PEDESTRIANS_KILLED'),
 (17, u'NUMBER_OF_PERSONS_INJURED'),
 (18, u'NUMBER_OF_PERSONS_KILLED'),
 (19, u'OFF_STREET_NAME'),
 (20, u'ON_STREET_NAME'),
 (21, u'UNIQUE_KEY'),
 (22, u'VEHICLE_TYPE_CODE_1'),
 (23, u'VEHICLE_TYPE_CODE_2'),
 (24, u'VEHICLE_TYPE_CODE_3'),
 (25, u'VEHICLE_TYPE_CODE_4'),
 (26, u'VEHICLE_TYPE_CODE_5'),
 (27, u'ZIP_CODE')]

In [4]:
list(enumerate(weather_data.first().split(',')))

[(0, u''),
 (1, u'Conditions'),
 (2, u'DateUTC'),
 (3, u'DatetimeEDT'),
 (4, u'Dew_PointF'),
 (5, u'Events'),
 (6, u'Foggy'),
 (7, u'Freezing'),
 (8, u'Gust_SpeedMPH'),
 (9, u'Humidity'),
 (10, u'PrecipitationIn'),
 (11, u'Rain'),
 (12, u'Sea_Level_PressureIn'),
 (13, u'Snow'),
 (14, u'TemperatureF'),
 (15, u'TimeEDT'),
 (16, u'VisibilityMPH'),
 (17, u'WindDirDegrees'),
 (18, u'Wind_Direction'),
 (19, u'Wind_SpeedMPH'),
 (20, u'Windy')]

In [36]:
import csv
import datetime
def map_weather_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour= p[3].split(':')[0]
        yield (datehour, p)

weather_data_rdd = weather_data.mapPartitionsWithIndex(map_weather_hours)
# sorted(weather_data_rdd.groupByKey().mapValues(list).collect())
weather_data_rdd = weather_data_rdd.groupByKey().mapValues(list)

In [35]:
import csv
from dateutil.parser import parse

def map_collision_hours(idx, part):
    if idx == 0:
        part.next()
    for p in csv.reader(part):
        datehour = str(parse(p[7])).split(':')[0]
        yield (datehour, p)

nypd_collisions_rdd = nypd_collisions.mapPartitionsWithIndex(map_collision_hours)
nypd_collisions_rdd.take(5)

[('2017-04-16 00',
  ['BRONX',
   'Turning Improperly',
   'Unspecified',
   '',
   '',
   '',
   'BRONX RIVER AVENUE',
   '04/16/2017 0:00:00',
   '40.831287',
   '(40.831287, -73.88173)',
   '-73.88173',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '',
   'EAST 172 STREET',
   '3651861',
   'PASSENGER VEHICLE',
   'PICK-UP TRUCK',
   '',
   '',
   '',
   '10472']),
 ('2017-04-16 00',
  ['BROOKLYN',
   'Turning Improperly',
   'Unspecified',
   '',
   '',
   '',
   'SMITH STREET',
   '04/16/2017 0:00:00',
   '40.686726',
   '(40.686726, -73.99047)',
   '-73.99047',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '',
   'BERGEN STREET',
   '3651519',
   'TAXI',
   'PASSENGER VEHICLE',
   '',
   '',
   '',
   '11201']),
 ('2017-04-16 00',
  ['BROOKLYN',
   'Unspecified',
   'Unspecified',
   '',
   '',
   '',
   '',
   '04/16/2017 0:00:00',
   '40.70256',
   '(40.70256, -73.99114)',
   '-73.99114',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',


In [40]:
joined_data = nypd_collisions_rdd.join(weather_data_rdd)

In [42]:
joined_data_collected = joined_data.collect()

In [43]:
joined_data_collected[0]

('2016-08-14 10',
 (['BRONX',
   'Unspecified',
   'Unspecified',
   '',
   '',
   '',
   'UNIVERSITY AVENUE',
   '08/14/2016 10:08:00',
   '',
   '',
   '',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '',
   'WEST FORDHAM ROAD',
   '3502375',
   'PASSENGER VEHICLE',
   'PASSENGER VEHICLE',
   '',
   '',
   '',
   '10468'],
  [['60271',
    'Clear',
    '2016-08-14 14:51:00',
    '2016-08-14 10:51:00',
    '72.0',
    '',
    '1',
    '0',
    '-',
    '57',
    'N/A',
    '0',
    '29.96',
    '0',
    '89.1',
    '10:51 AM',
    '9.0',
    '300',
    'WNW',
    '9.2',
    '0']]))

In [58]:
x = joined_data.take(1)[0]
datehour, tupled_lists = x[0], x[1]
tuple_weather = tupled_lists[1]
tuple_weather[0]
x[1][1][0]

['60271',
 'Clear',
 '2016-08-14 14:51:00',
 '2016-08-14 10:51:00',
 '72.0',
 '',
 '1',
 '0',
 '-',
 '57',
 'N/A',
 '0',
 '29.96',
 '0',
 '89.1',
 '10:51 AM',
 '9.0',
 '300',
 'WNW',
 '9.2',
 '0']

In [89]:
from pyspark.sql import Row
final_join=joined_data.map(lambda x: x[1][0] + x[1][1][0])

In [90]:
final_join_df = sqlContext.createDataFrame(final_join)

In [94]:
final_join_df.write.csv('joined_weather_collisions')

In [96]:
final_join_df.columns

['_1',
 '_2',
 '_3',
 '_4',
 '_5',
 '_6',
 '_7',
 '_8',
 '_9',
 '_10',
 '_11',
 '_12',
 '_13',
 '_14',
 '_15',
 '_16',
 '_17',
 '_18',
 '_19',
 '_20',
 '_21',
 '_22',
 '_23',
 '_24',
 '_25',
 '_26',
 '_27',
 '_28',
 '_29',
 '_30',
 '_31',
 '_32',
 '_33',
 '_34',
 '_35',
 '_36',
 '_37',
 '_38',
 '_39',
 '_40',
 '_41',
 '_42',
 '_43',
 '_44',
 '_45',
 '_46',
 '_47',
 '_48',
 '_49']

AttributeError: 'RDD' object has no attribute 'explain'